In [ ]:
# ============================================================
# Gravit Simulation — Colab Extended Demo
# - figures/ (png + cascade txt)
# - aggregated_results.json
# - aggregated_report_final.md (with embedded images + cascade blocks)
# - interactive slider view
# ============================================================

!pip -q install matplotlib ipywidgets

import json, random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from ipywidgets import interact, IntSlider
from IPython.display import display, Markdown

# ------------------------------------------------------------
# 0) CONFIG
# ------------------------------------------------------------
FIGURES_DIR = Path("figures")
FIGURES_DIR.mkdir(exist_ok=True)

SCENARIOS_PATH = Path("scenarios.json")
RESULTS_JSON_PATH = Path("aggregated_results.json")
REPORT_MD_PATH = Path("aggregated_report_final.md")

random.seed(42)
np.random.seed(42)

# ------------------------------------------------------------
# 1) SCENARIOS (replace with your real scenarios_catalog.json if you want)
# ------------------------------------------------------------
scenarios = [
    {"id": "S1", "type": "Network Trust", "description": "Initial network trust propagation scenario."},
    {"id": "S2", "type": "Consensus Test", "description": "Testing Gravit Consensus trust dynamics."},
    {"id": "S3", "type": "Quantum Layer", "description": "Simulating Gravit Quantum Platform trust propagation."}
]

SCENARIOS_PATH.write_text(json.dumps(scenarios, indent=2), encoding="utf-8")
print(f"[ok] wrote {SCENARIOS_PATH}")

# ------------------------------------------------------------
# 2) SIMULATION (stub — replace later with real trust engine)
# ------------------------------------------------------------
def run_simulation(scenario: dict) -> dict:
    """
    Stub simulation: creates plausible before/after stats.
    Replace this function with real trust propagation results.
    """
    base = random.uniform(0.55, 0.95)
    delta = random.uniform(-0.25, 0.10)

    avg_before = base
    avg_after  = max(0.0, min(1.0, base + delta))

    min_before = max(0.0, avg_before - random.uniform(0.05, 0.20))
    max_before = min(1.0, avg_before + random.uniform(0.05, 0.20))

    min_after  = max(0.0, avg_after - random.uniform(0.05, 0.20))
    max_after  = min(1.0, avg_after + random.uniform(0.05, 0.20))

    return {
        "scenario_id": scenario["id"],
        "type": scenario.get("type", "N/A"),
        "description": scenario.get("description", ""),
        "avg_before": float(avg_before),
        "avg_after": float(avg_after),
        "min_before": float(min_before),
        "max_before": float(max_before),
        "min_after": float(min_after),
        "max_after": float(max_after),
        "notes": f"Stub simulation for {scenario['id']} (replace with real engine outputs)."
    }

def generate_time_series(result: dict, steps: int = 20):
    t = np.linspace(0, 10, steps)
    y = result["avg_before"] + (result["avg_after"] - result["avg_before"]) * (t / t[-1])
    return t, y

# ------------------------------------------------------------
# 3) FIGURES: TRUST CHARTS (PNG) + CASCADE (TXT)
# ------------------------------------------------------------
def generate_trust_chart_png(sid: str, result: dict, out_dir: Path = FIGURES_DIR):
    t, y = generate_time_series(result)

    plt.figure(figsize=(7, 4))
    plt.plot(t, y, marker="o", label=f"{sid} avg trust")
    plt.fill_between(t, result["min_before"], result["max_before"], alpha=0.12, label="min/max before")
    plt.fill_between(t, result["min_after"],  result["max_after"],  alpha=0.12, label="min/max after")
    plt.title(f"Trust propagation — {sid}")
    plt.xlabel("time steps")
    plt.ylabel("trust")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    png_path = out_dir / f"{sid}_trust.png"
    plt.savefig(png_path)
    plt.close()
    return png_path

def generate_cascade_txt(sid: str, out_dir: Path = FIGURES_DIR):
    depth = random.randint(2, 6)
    lines = ["Root Node"]
    for i in range(1, depth + 1):
        lines.append(("  " * i) + f"-> Node {i}")
    txt_path = out_dir / f"{sid}_cascade.txt"
    txt_path.write_text("\n".join(lines), encoding="utf-8")
    return txt_path

# ------------------------------------------------------------
# 4) REPORT: JSON + MARKDOWN (with embedded image links + cascade blocks)
# ------------------------------------------------------------
def export_results_json(results: dict, path: Path = RESULTS_JSON_PATH):
    path.write_text(json.dumps(results, indent=2), encoding="utf-8")
    print(f"[ok] wrote {path}")

def export_markdown_report(results: dict, path: Path = REPORT_MD_PATH, figures_dir: Path = FIGURES_DIR):
    today = datetime.utcnow().strftime("%Y-%m-%d")
    md = []
    a = md.append

    a("# Gravit Simulation — Aggregated Report")
    a(f"Date: {today}")
    a("")
    a("This report is auto-generated in Colab. Replace stub simulation with real Gravit trust-engine outputs when ready.")
    a("")

    # Summary table
    a("## 1. Summary Table")
    a("")
    a("| Scenario ID | Type | Description | Avg Before | Avg After | Min/Max Before | Min/Max After | Graph |")
    a("|---|---|---|---:|---:|---:|---:|---|")

    for sid, r in results.items():
        png = figures_dir / f"{sid}_trust.png"
        graph_md = f"![{sid}]({png.as_posix()})" if png.exists() else "N/A"
        a(
            f"| {sid} | {r.get('type','N/A')} | {r.get('description','')} | "
            f"{r['avg_before']:.4f} | {r['avg_after']:.4f} | "
            f"{r['min_before']:.4f}/{r['max_before']:.4f} | "
            f"{r['min_after']:.4f}/{r['max_after']:.4f} | "
            f"{graph_md} |"
        )

    a("")
    a("## 2. Scenario Details")
    a("")

    for sid, r in results.items():
        a(f"### Scenario {sid}")
        a(f"**Type:** {r.get('type','N/A')}")
        a(f"**Description:** {r.get('description','')}")
        a("")
        a("**Pre-run metrics**")
        a(f"- avg: {r['avg_before']:.4f}")
        a(f"- min/max: {r['min_before']:.4f} / {r['max_before']:.4f}")
        a("")
        a("**Post-run metrics**")
        a(f"- avg: {r['avg_after']:.4f}")
        a(f"- min/max: {r['min_after']:.4f} / {r['max_after']:.4f}")
        a("")
        a("**Notes**")
        a(f"- {r.get('notes','')}")
        a("")

        png = figures_dir / f"{sid}_trust.png"
        if png.exists():
            a(f"![Trust Chart {sid}]({png.as_posix()})")
            a("")

        txt = figures_dir / f"{sid}_cascade.txt"
        if txt.exists():
            a("**Cascade (text)**")
            a("")
            a("```")
            a(txt.read_text(encoding="utf-8"))
            a("```")
            a("")

    path.write_text("\n".join(md), encoding="utf-8")
    print(f"[ok] wrote {path}")

# ------------------------------------------------------------
# 5) RUN PIPELINE (simulate -> figures -> exports)
# ------------------------------------------------------------
results = {sc["id"]: run_simulation(sc) for sc in scenarios}

for sid, r in results.items():
    generate_trust_chart_png(sid, r)
    generate_cascade_txt(sid)

export_results_json(results, RESULTS_JSON_PATH)
export_markdown_report(results, REPORT_MD_PATH, FIGURES_DIR)

print(f"[ok] figures dir: {FIGURES_DIR.resolve()}")
print(f"[ok] report: {REPORT_MD_PATH.resolve()}")
print(f"[ok] results json: {RESULTS_JSON_PATH.resolve()}")

# ------------------------------------------------------------
# 6) INTERACTIVE VIEW (slider)
# ------------------------------------------------------------
scenario_ids = list(results.keys())

def interactive_plot(idx: int):
    sid = scenario_ids[idx]
    r = results[sid]
    t, y = generate_time_series(r)

    plt.figure(figsize=(7, 4))
    plt.plot(t, y, marker="o", label=f"{sid} avg trust")
    plt.fill_between(t, r["min_before"], r["max_before"], alpha=0.12, label="min/max before")
    plt.fill_between(t, r["min_after"],  r["max_after"],  alpha=0.12, label="min/max after")
    plt.title(f"Interactive trust — {sid}")
    plt.xlabel("time steps")
    plt.ylabel("trust")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend()
    plt.show()

    cascade_path = FIGURES_DIR / f"{sid}_cascade.txt"
    print("Cascade (text):")
    print(cascade_path.read_text(encoding="utf-8") if cascade_path.exists() else "N/A")
    print("\nNotes:", r.get("notes",""))
    print("Type:", r.get("type","N/A"))
    print("Description:", r.get("description",""))

print("\n[interactive] use slider below")
interact(interactive_plot, idx=IntSlider(min=0, max=len(scenario_ids)-1, step=1, description="scenario"))

# ------------------------------------------------------------
# 7) OPTIONAL: preview markdown inside Colab
# ------------------------------------------------------------
display(Markdown(REPORT_MD_PATH.read_text(encoding="utf-8")))

# ------------------------------------------------------------
# 8) OPTIONAL: download files from Colab
# ------------------------------------------------------------
# from google.colab import files
# files.download(str(RESULTS_JSON_PATH))
# files.download(str(REPORT_MD_PATH))
# # Zip figures for download:
# import shutil
# shutil.make_archive("figures", "zip", FIGURES_DIR)
# files.download("figures.zip")


: 